In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Install

In [22]:
!pip install transformers
!pip install fastBPE
!pip install fairseq

# Install the vncorenlp python wrapper
!pip install py_vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 25.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483860 sha256=ab0c9170006ca9726b2195f5b4e973e5f41b40e3c3092535a3e79d94b65ffe81
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████

#Import

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from torch import nn
import json
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import torch
import matplotlib.pyplot as plt
import torch.utils.data
import torch.nn.functional as F
import argparse
from transformers import *
from transformers.modeling_utils import * 
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary

2022-08-01 08:49:29 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [2]:
import py_vncorenlp
# Load the word and sentence segmentation component
# rdrsegmenter = VnCoreNLP(args.rdrsegmenter_path, annotators="wseg", max_heap_size='-Xmx500m') 
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/drive/MyDrive/AICamp/Hackathon/vncorenlp')

#Datasets

##Kaggle

In [3]:
# !pip install --upgrade --no-cache-dir gdown
# !gdown --id 1-hCedCKQkgRv8dPyymFpIcnIaHoPigBT

##Drive

In [4]:
base_dir = '/content/drive/MyDrive/AICamp/Hackathon'
train_path = os.path.join(base_dir, 'Datasets/datasets.csv')

In [5]:
train_df = pd.read_csv(train_path)

In [6]:
train_df.head()

,Review,giai_tri,luu_tru,nha_hang,an_uong,di_chuyen,mua_sam
0,"Bánh rất nhiều tôm to, tôm giòn nằm chễm chệ t...",0,0,0,5,0,0
1,Đi Quy Nhơn cứ lo không có pizza ăn nhưng nhờ ...,0,0,0,5,0,0
2,"các món ăn ở đây không có gì đặc sắc, có lẽ nấ...",0,0,1,1,0,0
3,"Lần thứ 2 quay lại đây, vì giới thiệu quán cà ...",0,0,0,2,0,0
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0


#Preprocess 

##Label

In [7]:
dict_label = {0: [0,0,0,0,0], 1:[1,0,0,0,0], 2:[0,1,0,0,0], 3:[0,0,1,0,0], 4:[0,0,0,1,0], 5:[0,0,0,0,1]}

In [8]:
dict_label

{0: [0, 0, 0, 0, 0],
 1: [1, 0, 0, 0, 0],
 2: [0, 1, 0, 0, 0],
 3: [0, 0, 1, 0, 0],
 4: [0, 0, 0, 1, 0],
 5: [0, 0, 0, 0, 1]}

In [9]:
aspects_name = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "di_chuyen", "mua_sam"] 

labels_train = []
for i in range(len(train_df['Review'])):
    label = []
    for name in aspects_name:
        for x in dict_label:
            if train_df[name][i] == x:
                label += dict_label[x]
    labels_train.append(np.array(label))

train_df.rename(columns = {'Review':'text'}, inplace = True)
train_df['label'] = labels_train

In [10]:
train_df.head()

,text,giai_tri,luu_tru,nha_hang,an_uong,di_chuyen,mua_sam,label
0,"Bánh rất nhiều tôm to, tôm giòn nằm chễm chệ t...",0,0,0,5,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Đi Quy Nhơn cứ lo không có pizza ăn nhưng nhờ ...,0,0,0,5,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"các món ăn ở đây không có gì đặc sắc, có lẽ nấ...",0,0,1,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,"Lần thứ 2 quay lại đây, vì giới thiệu quán cà ...",0,0,0,2,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


#Model

In [11]:
class RobertaForAIViVN(BertPreTrainedModel):
   config_class = RobertaConfig
   base_model_prefix = "roberta"
   def __init__(self, config):
       super(RobertaForAIViVN, self).__init__(config)
       self.num_labels = config.num_labels
       self.roberta = RobertaModel(config)
       self.qa_outputs = nn.Linear(4*config.hidden_size, self.num_labels)

       self.init_weights()

   def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):

       outputs = self.roberta(input_ids,
                            attention_mask=attention_mask,
#                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)
       cls_output = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
       logits = self.qa_outputs(cls_output)
       return logits


#Utils

In [12]:
def convert_lines(df, vocab, bpe, max_sequence_length):
    outputs = np.zeros((len(df), max_sequence_length))
    
    cls_id = 0
    eos_id = 2
    pad_id = 1

    for idx, row in tqdm(df.iterrows(), total=len(df)): 
        subwords = bpe.encode('<s> '+row.text+' </s>')
        input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
        if len(input_ids) > max_sequence_length: 
            input_ids = input_ids[:max_sequence_length] 
            input_ids[-1] = eos_id
        else:
            input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
        outputs[idx,:] = np.array(input_ids)
    return outputs

def seed_everything(SEED):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


#Metrics

In [14]:
pred = [[1, 0, 3, 4, 2, 0], [0, 2, 2, 5, 1, 0]]
gt = [[1, 1, 3, 5, 3, 1], [0, 0, 3, 4, 0, 0]]

In [15]:
def my_r2_score(pred, gt):
    result = []
    for i in range(pred.shape[1]):
        a = pred[:,i]
        b = gt[:,i]
        RSS = 0
        K = 0
        for j in range(a.shape[0]):
            RSS += (a[j] - b[j])**2
            K += 16

        result.append(1 - (RSS/K))

    return np.array(result)

In [16]:
def metrics(pred, gt):
    pred = np.array(pred, dtype='float32')
    gt = np.array(gt, dtype='float32')
    
    bool_pred = pred > 0
    bool_gt = gt > 0

    f1s = f1_score(bool_pred, bool_gt, average=None)
    r2s = my_r2_score(pred, gt)

    result = np.mean(f1s * r2s)
    
    return result

In [17]:
metrics(pred, gt)

0.5885416666666666

#Train

In [20]:
max_sequence_length = 256
epochs = 5
batch_size = 24
accumulation_steps = 5
lr = 3e-5
ckpt_path = "/content/models"
fold = 0

# Code
parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/drive/MyDrive/AICamp/Hackathon/PhoBERT/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)



seed_everything(69)


# Load model
'''
config = RobertaConfig.from_pretrained(
    args.config_path,
    output_hidden_states=True,
    num_labels=1
)
model_bert = RobertaForAIViVN.from_pretrained(args.pretrained_path, config=config)
model_bert.cuda()
'''

config = RobertaConfig.from_pretrained(
    "/content/drive/MyDrive/AICamp/Hackathon/PhoBERT/PhoBERT_base_transformers/config.json", num_labels = 30, output_hidden_states=True,
)
model_bert = RobertaForAIViVN.from_pretrained(
    "/content/drive/MyDrive/AICamp/Hackathon/PhoBERT/PhoBERT_base_transformers/model.bin",
    config=config
)
model_bert.cuda()


if torch.cuda.device_count():
    print(f"Training using {torch.cuda.device_count()} gpus")
    model_bert = nn.DataParallel(model_bert)
    tsfm = model_bert.module.roberta
else:
    tsfm = model_bert.roberta

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/AICamp/Hackathon/PhoBERT/PhoBERT_base_transformers/dict.txt")

# Load training data
train_df.text = train_df.text.progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.word_segment(x)]))
y = train_df.label.values
y = np.stack(y).astype(None)

X_train = convert_lines(train_df, vocab, bpe, max_sequence_length)

# Creating optimizer and lr schedulers
param_optimizer = list(model_bert.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

num_train_optimization_steps = int(epochs * len(train_df) / batch_size / accumulation_steps)
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)  # PyTorch scheduler
scheduler0 = get_constant_schedule(optimizer)  # PyTorch scheduler

if not os.path.exists(ckpt_path):
    os.mkdir(ckpt_path)

z = train_df.giai_tri.values
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X_train, z))
for fold, (train_idx, val_idx) in enumerate(splits):
    print("Training for fold {}".format(fold))
    best_score = 0
    if fold != fold:
        continue
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train[train_idx],dtype=torch.long), torch.tensor(y[train_idx],dtype=torch.long))
    valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train[val_idx],dtype=torch.long), torch.tensor(y[val_idx],dtype=torch.long))
    tq = tqdm(range(epochs + 1))
    for child in tsfm.children():
        for param in child.parameters():
            if not param.requires_grad:
                print("whoopsies")
            param.requires_grad = False
    frozen = True
    for epoch in tq:

        if epoch > 0 and frozen:
            for child in tsfm.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
            # del scheduler0
            torch.cuda.empty_cache()

        val_preds = None
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
        avg_loss = 0.
        avg_accuracy = 0.

        optimizer.zero_grad()
        pbar = tqdm(enumerate(train_loader),total=len(train_loader),leave=False)
        for i,(x_batch, y_batch) in pbar:
            model_bert.train()
            y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
            loss =  F.binary_cross_entropy_with_logits(torch.flatten(y_pred).view(-1).cuda(),torch.flatten(y_batch).float().cuda())
            loss = loss.mean()
            loss.backward()
            if i % accumulation_steps == 0 or i == len(pbar) - 1:
                optimizer.step()
                optimizer.zero_grad()
                if not frozen:
                    scheduler.step()
                else:
                    scheduler0.step()
            lossf = loss.item()
            pbar.set_postfix(loss = lossf)
            avg_loss += loss.item() / len(train_loader)

        model_bert.eval()
        pbar = tqdm(enumerate(valid_loader),total=len(valid_loader),leave=False)
        for i,(x_batch, y_batch) in pbar:
            y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
            y_pred = y_pred.squeeze().detach().cpu().numpy()
            val_preds = np.atleast_1d(y_pred) if val_preds is None else np.concatenate([val_preds, np.atleast_1d(y_pred)])
        val_preds = sigmoid(val_preds)

        best_th = 0
        score = f1_score(y[val_idx], val_preds > 0.5, average=None)
        score = score.mean()
        print(f"F1 score @0.5 = {score:.4f}")
        if score >= best_score:
            torch.save(model_bert.state_dict(),os.path.join(ckpt_path, f"model_{fold}.bin"))
            best_score = score


loading configuration file /content/drive/MyDrive/AICamp/Hackathon/PhoBERT/PhoBERT_base_transformers/config.json
You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": 

Training using 1 gpus


 36%|███▋      | 1343/3688 [00:03<00:05, 433.31it/s]


KeyboardInterrupt: ignored

In [ ]:
test_df = pd.read_csv(args.test_path,sep='\t').fillna("###")
test_df.text = test_df.text.progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(x)]))
X_test = convert_lines(test_df, vocab, bpe,args.max_sequence_length)

preds_en = []
for fold in range(5):
    print(f"Predicting for fold {fold}")
    preds_fold = []
    model_bert.load_state_dict(torch.load(os.path.join(args.ckpt_path, f"model_{fold}.bin")))
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test,dtype=torch.long))
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    model_bert.eval()
    pbar = tqdm(enumerate(test_loader),total=len(test_loader),leave=False)
    for i, (x_batch,) in pbar:
        y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
        y_pred = y_pred.view(-1).detach().cpu().numpy()
        preds_fold = np.concatenate([preds_fold, y_pred])
    preds_fold = sigmoid(preds_fold)
    preds_en.append(preds_fold)
preds_en = np.mean(preds_en,axis=0)
test_df["label"] = (preds_en > 0.5).astype(np.int)
test_df[["id","label"]].to_csv("submission.csv")


In [ ]:
# Load valid data
df_valid.text = df_valid.text.progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.word_segment(x)]))
y = df_valid.label.values
y = np.stack(y).astype(None)

X_test = convert_lines(df_valid, vocab, bpe, max_sequence_length)

preds_en = []
print(f"Predicting")
preds_fold = []

test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test,dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
model_bert.eval()
pbar = tqdm(enumerate(test_loader),total=len(test_loader),leave=False)
for i, (x_batch,) in pbar:
    y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
    y_pred = y_pred.view(-1).detach().cpu().numpy()
    preds_fold = np.concatenate([preds_fold, y_pred])
preds_fold = sigmoid(preds_fold)
preds_en.append(preds_fold)
# preds_en = np.mean(preds_en,axis=0)
# preds_en = (preds_en > 0.5).astype(np.int)

100%|██████████| 970/970 [00:01<00:00, 674.02it/s]


Predicting


In [ ]:
print(preds_en)

[array([0.02638777, 0.01358304, 0.01176553, ..., 0.02413399, 0.03635803,
       0.03328108])]


In [ ]:
a = preds_en[0].reshape([970,30])

In [ ]:
a = np.round(a, 2)

In [ ]:
df_valid["predict"] = list(a)

In [ ]:
df_valid.head()

,text,giai_tri,luu_tru,nha_hang,an_uong,di_chuyen,mua_sam,label,predict
0,N ơ i đ ó t h ậ t k i n ...,0,0,1,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.03, 0.01, 0.01, 0.05, 0.09, 0.02, 0.02, 0.0..."
1,"Q u á n s ạ c h _ s ẽ , ...",0,0,0,5,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.03, 0.02, 0.02, 0.06, 0.09, 0.02, 0.02, 0.0..."
2,P h ụ c _ v ụ m ã i _ m ...,0,0,0,2,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.02, 0.01, 0.01, 0.03, 0.08, 0.01, 0.02, 0.0..."
3,N h i ề u m ó n đ ă c t ...,0,0,0,5,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.03, 0.02, 0.01, 0.06, 0.09, 0.02, 0.02, 0.0..."
4,K h ô n g p h ả i l à m ...,0,0,0,0,2,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.02, 0.01, 0.01, 0.04, 0.08, 0.01, 0.02, 0.0..."


In [ ]:
df_valid.to_csv("/content/result/submission.csv")